In [ ]:
# https://www.visualcapitalist.com/the-history-of-wearable-technology/

In [1]:
# !pip install --upgrade pip
# !pip install --upgrade pandas
# !pip install --upgrade seaborn
# !pip install --upgrade scipy
# !pip install psycopg2-binary

## Adoption life cycle
The cumulative number of patents plotted over time follows an s-curve with discernable life cycle phases. This is imprecise alone but useful as an overview. If a technology isn’t in its final phase, the s-curve isn’t clearly visible which makes identifying a life cycle more difficult.

## CPC Diversity
Continuously decreasing values (negative trend line) indicates that the variety of technological alternatives is slowly consolidating towards a dominationg field.

## CPC concentration
If a single classification is beginning to dominate the others (>50%), the overall variety is expected to decline until one category ultimately dominates all others. This gives way to dicontinuity/decline.

## CPC Recombination
A patent has precedence when it has at least one pair of CPC groups that were previously unconnected. As a field stagnates, new inventions are more often the results of recombination of existing technologies.

In [2]:
# Jupyter magics to make life easier
# View plots in adjacent cell
%matplotlib inline

%load_ext autoreload

%autoreload 2

from precedence import prima_facie, pf_visualizations

import matplotlib.pyplot as plt
import seaborn as sns

# Uncomment one of the pairs for a test case or add your own

In [3]:
# uspto_patent_id = "US-6619835-B2" # casio wearable
# seed_name = "casio"

# uspto_patent_id = "US-7128988-B2" # magnetic materials
# seed_name = "magmat"

# uspto_patent_id = "US-7640837-B2" # sawstop
# seed_name = "sawstop"

# uspto_patent_id = "US-7743213-B2" # nas
# seed_name = "nas"

# uspto_patent_id = "US-9339713-B2" # nike golf hud
# seed_name = "nike"

# uspto_patent_id = "US-20160378108-A1" # amazon drone collective
# seed_name = "amazon_drone"

# Seawater JetFuel
# uspto_patent_id = "US-9303323-B2"
# seed_name = "seawater_jetfuel"

# Nokia mobile phone
uspto_patent_id = "US-20020028693-A1"
seed_name = "nokia"

# uspto_patent_id = "US-20060094518-A1"
# seed_name = "butt_kicking"

# PCR
# US-4683202-A
# uspto_patent_id = "US-4683202-A"
# seed_name = "pcr"

# 3DPrinting
# US-4575330-A
# uspto_patent_id = "US-4575330-A"
# seed_name = "3dprinting"

# LabOnAChip
# US-5858195-A 
# uspto_patent_id = "US-5858195-A"
# seed_name = "labonachip"

# CRISPR (this is an MIT patent which is in litigation with UC Berkeley who says they were first)
# US-8697359-B1
# uspto_patent_id = "US-8697359-B1"
# seed_name = "crisper"

# uspto_patent_id = "US-6139035-A"
# seed_name = "scooter_brake"

# uspto_patent_id = "US-9051066-B1"
# seed_name = "bunch_o_balloons"

# uspto_patent_id = "US-5960411-A"
# seed_name = "amazon_sales"

# uspto_patent_id = "US-8527057-B2"
# seed_name = "bionic_eye"

# uspto_patent_id = "US-7071258-B1"
# seed_name = "graphene"

# uspto_patent_id = "US-6618592-B1"
# seed_name = "3g"

# uspto_patent_id = "US-20070291130-A1"
# seed_name = "autonomous_vehicle_vision"

# uspto_patent_id = "US-6073115-A"
# seed_name = "vr_generator"

In [4]:
# If you haven't used the similarity console to download similar patents then change get_similar to True
# and this sim_pats_file to be the location you want to save the zip file
sim_pats_file = f"{seed_name}.zip"

result = prima_facie.get_sim_patents(seed_pat_id=uspto_patent_id,
                                                sim_pats_file = sim_pats_file,
                                                get_similar=True)

Logging into OKTA
Login Success
Requesting Similarity Job
{'job_id': 'ab94fdb6-fbe5-4f60-8dff-2be9a383729e'}
Job started, job_id = ab94fdb6-fbe5-4f60-8dff-2be9a383729e
Waiting for Job, jobs take about 2 minutes
Job Complete
Checking Results
Results file downloaded to nokia.zip


In [6]:
indicators, data, section_df, seed_year = prima_facie.get_indicators(sim_df=result, seed_patent=uspto_patent_id)

score = 1 + sum(indicators.values())

print(score)

5.0


In [7]:
seed_year

2000

# How we extract segments of increasing and decreasing trend in cpc diversity

In [ ]:
# data, section_df = prima_facie.agg_yearly(result, uspto_patent_id)

In [ ]:
seed = prima_facie.get_priority_date(patent_ids=uspto_patent_id, single=True)
priority_yr = seed['priority_dt'][0].year

In [ ]:
df = data.sort_values('priority_yr', ascending=True)
all_years = data['priority_yr'].unique()
columns_to_norm = ['cpc_diversity']
for i in columns_to_norm:
    df[f"norm_{i}"] = (df[i]-df[i].min())/(df[i].max()-df[i].min())
x = df['norm_cpc_diversity'].values
labels = prima_facie.label_status(x)
seg_list, idx_list = prima_facie.bottom_up(x, labels, 0.03, k=1, step=4)
segments = prima_facie.combine_segment(x, labels, seg_list, idx_list)

# Plot the segments
plt.figure(figsize=(16, 4))
status_color = ['c', 'black', 'gray']
plt.plot(range(len(x)), x, '-', label='Normalized CPC Diversity')
for t in range(len(x)):
    plt.plot([t], [x[t]], marker='o', markersize=5, c=status_color[int(labels[t])])

for seg, idx in zip(seg_list, idx_list):
    line, slope = prima_facie.approximate_line(seg)
    plt.plot(range(idx[0], idx[1]), line, '--', linewidth=2, c='c' if slope > 0 else 'gray')

for seg in segments:
    if seg[2] == prima_facie.INC:
        plt.plot([seg[0], seg[0]], [0, 1], ':', color='c', label='upward segment')
        plt.plot([seg[1]-1, seg[1]-1], [0, 1], ':', color='c')
    elif seg[2] == prima_facie.DEC:
        plt.plot([seg[0], seg[0]], [0, 1], ':', color='gray', label='downward segment')
        plt.plot([seg[1]-1, seg[1]-1], [0, 1], ':', color='gray')

plt.xticks(range(len(all_years)), all_years)
plt.legend(loc='upper right')

# Graphical analysis

In [ ]:
result.loc[result['priority_yr'] > priority_yr, 'priority'] = 'after'
result.loc[result['priority_yr'] <= priority_yr, 'priority'] = 'before'
fig, ax = plt.subplots(figsize=(26,12))
sns.countplot(x='priority_yr', data=result, hue='priority', ax=ax)

# Adoption S-curve

The cumulative number of patents plotted over time follows an s-curve with discernable life cycle phases. This visual is imprecise alone but useful as an overview. If a technology isn’t in its final phase, the s-curve isn’t clearly visible which makes identifying a life cycle more difficult. 

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sns.lineplot(x='priority_yr', y='applications_cumsum', data=data, hue='adoption_quartiles')
plt.savefig(f"{seed_name}_adoption.png")


# Trend in similarity

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sns.lineplot(x='priority_yr', y='cos_sim', data=data, hue='priority')
plt.savefig(f"{seed_name}_similarity.png")


# Section concentration

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sns.lineplot(x='priority_yr', y='section_concentration', data=data, hue='priority')
plt.savefig(f"{seed_name}_concentration.png")

# CPC diversity

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
sns.lineplot(x='priority_yr', y='cpc_diversity', data=data, hue='priority')
plt.savefig(f"{seed_name}_diversity.png")

# Technological changes

In [ ]:
pf_visualizations.create_timeline(data=data,
                    x="applications",
                    y="applicants",
                    size="applicants",
                    hue="priority_yr",
                    year_col="priority_yr",
                    seed_year=priority_yr,
                    highlight_seed_year=True,
                    savefig=True,
                    showplot=True,
                    fig_filename=f"{seed_name}_tech_changes.png"
                   )

# Stages of maturity

In [ ]:
pf_visualizations.create_timeline(data=data,
                    x="applications",
                    y="delta_applications",
                    size="applicants",
                    hue="priority_yr",
                    year_col="priority_yr",
                    highlight_seed_year=True,
                    seed_year=priority_yr,
                    savefig=True,
                    showplot=True,
                    fig_filename=f"{seed_name}_delta_apps.png"
                   )

# Section Concentration

In [ ]:
all_years = section_df['year'].unique()
for year in all_years:
    cpc1_count = section_df['application_count'][section_df['year']==year]
    cpc1_total = section_df['application_count'][section_df['year']==year].sum()
    section_df.loc[section_df['year']==year,"section_concentration"] = cpc1_count/cpc1_total

import pandas as pd
new_cpc1 = pd.pivot_table(section_df,
                index=["year"],
                values=["section_concentration"],
                columns=["section"],
                aggfunc=['sum'],
                fill_value=0)

colors=sns.color_palette("Paired")
new_cpc1.plot.bar(stacked=True, figsize=(10,8),color=colors, legend=False)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))


In [ ]:
fig, ax = plt.subplots(figsize=(24,6))
sns.barplot(x='priority_yr', y='section_concentration', data=data)
plt.savefig(f"{seed_name}_cpc_concentration.png")